# 🚀 FX Sentiment Analysis: 15-Year Pipeline on Google Colab

Complete end-to-end pipeline: **Bronze Ingestion → Silver Features → Gold Matrix → Training → Evaluation**

**Expected Runtime**: 2-4 hours for 15 years of data  
**Data Volume**: ~500MB-2GB depending on sources  
**Models**: Logistic Regression, GBT, XGBoost (with fallback)  

---

In [ ]:
# Cell 1: Install Dependencies (Runtime: ~3-5 minutes)
print("📦 Installing Java 17, Spark 3.5.0, Delta Lake, XGBoost4J-Spark...")

!apt-get update -qq && apt-get install -y openjdk-17-jdk-headless -qq

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'

!pip install pyspark==3.5.0 delta-spark==3.0.0 mlflow==2.8.1 -q
!pip install scikit-learn==1.3.2 xgboost==2.0.3 plotly==5.17.0 -q
!pip install pandas numpy requests python-dotenv pyarrow pyyaml tqdm matplotlib -q

print("✅ Dependencies installed successfully!")

In [ ]:
# Cell 2: Setup Repository (Runtime: ~1 minute)
print("📂 Setting up repository...")

!git clone https://github.com/maadhusn/CN7030---Hamad-Hussain.git /content/fx-pipeline
%cd /content/fx-pipeline/fx-sentiment-pyspark
!git checkout colab-15y-run-ready

import sys
sys.path.append('/content/fx-pipeline/fx-sentiment-pyspark')

print("✅ Repository setup complete!")
print(f"📍 Working directory: {os.getcwd()}")

In [ ]:
# Cell 3: Configure API Keys
print("🔑 Configuring API keys...")

API_KEYS = {
    'ALPHAVANTAGE_API_KEY': 'your_alpha_vantage_key_here',  # ⚠️ REPLACE
    'TWELVEDATA_API_KEY': 'your_twelvedata_key_here',       # ⚠️ REPLACE  
    'FRED_API_KEY': 'your_fred_key_here',                   # ⚠️ REPLACE
    'TRADINGECONOMICS_API_KEY': '',                         # Optional
}

missing_keys = [k for k, v in API_KEYS.items() if not v and k != 'TRADINGECONOMICS_API_KEY']
if missing_keys:
    print(f"❌ Missing required API keys: {missing_keys}")
    print("Please update the API_KEYS dictionary above with your actual keys.")
else:
    with open('/content/fx-pipeline/fx-sentiment-pyspark/conf/secrets.env', 'w') as f:
        for key, value in API_KEYS.items():
            f.write(f"{key}={value}\n")
        f.write("BIG_RUN=true\n")
        f.write("ALLOW_BIG_RUN=true\n")
        f.write("MAX_TRAIN_ROWS=\n")  # No limit
        f.write("DELTA_BASE=/content/delta\n")
        f.write("ARTIFACTS_DIR=/content/artifacts\n")
    
    for key, value in API_KEYS.items():
        os.environ[key] = value
    
    print("✅ API keys configured successfully!")

In [ ]:
# Cell 4: Configure Pipeline (15-Year vs Smoke Test)
from datetime import datetime, timedelta

SMOKE_TEST = False  # Set to True for quick validation

if SMOKE_TEST:
    print("🧪 SMOKE TEST MODE: 90-day validation run")
    end_date = datetime.utcnow().date()
    start_date = end_date - timedelta(days=90)
    os.environ['MAX_TRAIN_ROWS'] = '10000'
else:
    print("🚀 FULL PIPELINE MODE: 15-year big data run")
    end_date = datetime.utcnow().date()
    start_date = end_date.replace(day=1) - timedelta(days=365*15)
    os.environ['MAX_TRAIN_ROWS'] = ''  # No limit

os.environ['START_DATE'] = start_date.isoformat()
os.environ['END_DATE'] = end_date.isoformat()
os.environ['BIG_RUN'] = 'true'
os.environ['ALLOW_BIG_RUN'] = 'true'

print(f"📅 Date Range: {start_date} to {end_date}")
print(f"📊 Data Volume: {'~50MB (smoke)' if SMOKE_TEST else '~500MB-2GB (full)'}")
print(f"⏱️ Expected Runtime: {'~15-30 min' if SMOKE_TEST else '~2-4 hours'}")

In [ ]:
# Cell 5: Initialize Spark Session
print("⚡ Initializing Spark with Delta Lake and XGBoost4J-Spark...")

from pyspark.sql import SparkSession

!mkdir -p /content/delta /content/models /content/reports/colab /content/artifacts

spark = SparkSession.builder \
    .appName("FX-Sentiment-15Y-Pipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "/content/delta") \
    .config("spark.sql.streaming.checkpointLocation", "/content/checkpoints") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0,ml.dmlc:xgboost4j-spark_2.12:2.0.3") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark {spark.version} initialized successfully!")
print(f"📂 Delta warehouse: /content/delta")
print(f"💾 Checkpoints: /content/checkpoints")

In [ ]:
# Cell 6: Bronze Data Ingestion (Runtime: 30-60 min for full, 5-10 min for smoke)
print("🥉 Starting Bronze Data Ingestion...")

import subprocess
import time

def run_ingestion_job(module_name, description):
    print(f"\n📥 {description}...")
    start_time = time.time()
    try:
        result = subprocess.run(
            ['python', '-m', module_name],
            cwd='/content/fx-pipeline/fx-sentiment-pyspark',
            capture_output=True,
            text=True,
            timeout=3600  # 1 hour timeout
        )
        elapsed = time.time() - start_time
        if result.returncode == 0:
            print(f"✅ {description} completed in {elapsed:.1f}s")
        else:
            print(f"❌ {description} failed: {result.stderr}")
            return False
    except subprocess.TimeoutExpired:
        print(f"⏰ {description} timed out after 1 hour")
        return False
    except Exception as e:
        print(f"❌ {description} error: {e}")
        return False
    return True

ingestion_jobs = [
    ('code_spark.ingest_fx', 'FX Data Ingestion'),
    ('code_spark.ingest_fred', 'FRED Economic Data Ingestion'),
    ('code_spark.ingest_gdelt_gkg', 'GDELT News Sentiment Ingestion'),
    ('code_spark.ingest_calendar_us', 'US Economic Calendar Ingestion'),
]

bronze_success = True
for module, desc in ingestion_jobs:
    if not run_ingestion_job(module, desc):
        bronze_success = False
        break

if bronze_success:
    print("\n🎉 Bronze ingestion completed successfully!")
    print("📊 Delta tables created: bronze_fx, bronze_fred_*, bronze_gkg, bronze_calendar_us")
else:
    print("\n❌ Bronze ingestion failed. Check logs above.")

In [ ]:
# Cell 7: Silver Feature Engineering (Runtime: 20-40 min for full, 3-5 min for smoke)
print("🥈 Starting Silver Feature Engineering...")

if bronze_success:
    silver_jobs = [
        ('code_spark.silver_fx_features', 'FX Technical Indicators'),
        ('code_spark.silver_us_calendar_1h', 'Economic Calendar Features'),
    ]
    
    silver_success = True
    for module, desc in silver_jobs:
        if not run_ingestion_job(module, desc):
            silver_success = False
            break
    
    if silver_success:
        print("\n🎉 Silver feature engineering completed!")
        print("📊 Features: Technical indicators, macro features, sentiment scores")
    else:
        print("\n❌ Silver feature engineering failed.")
else:
    print("⏭️ Skipping silver features due to bronze ingestion failure.")
    silver_success = False

In [ ]:
# Cell 8: Gold Training Matrix & Model Training (Runtime: 60-120 min for full, 10-15 min for smoke)
print("🥇 Creating Gold Training Matrix & Training Models...")

if silver_success:
    gold_success = run_ingestion_job('code_spark.gold_training_matrix', 'Gold Training Matrix Creation')
    
    if gold_success:
        print("\n🤖 Starting Model Training...")
        training_success = run_ingestion_job('code_spark.train_big', 'Model Training (LR, GBT, XGBoost)')
        
        if training_success:
            print("\n🎉 Training completed successfully!")
            print("📊 Models trained: Logistic Regression, GBT, XGBoost (with fallback)")
            print("💾 Models saved to: /content/models/")
        else:
            print("\n❌ Model training failed.")
    else:
        print("\n❌ Gold matrix creation failed.")
        training_success = False
else:
    print("⏭️ Skipping gold matrix and training due to silver feature failure.")
    training_success = False

In [ ]:
# Cell 9: Generate Final Pipeline Report
print("📋 Generating Final Pipeline Report...")

import json
from datetime import datetime

report = {
    "pipeline_execution": {
        "timestamp": datetime.utcnow().isoformat(),
        "mode": "smoke_test" if SMOKE_TEST else "full_15y_run",
        "date_range": {
            "start": os.environ.get('START_DATE'),
            "end": os.environ.get('END_DATE')
        },
        "stages": {
            "bronze_ingestion": bronze_success,
            "silver_features": silver_success,
            "gold_matrix_training": training_success
        }
    },
    "outputs": {
        "models": "/content/models/",
        "metrics": "/content/reports/colab/",
        "plots": "/content/reports/colab/",
        "delta_tables": "/content/delta/"
    },
    "anti_leak_safeguards": {
        "chronological_splits": True,
        "proper_time_windows": True,
        "left_only_joins": True,
        "event_windows_aligned": True
    }
}

with open('/content/reports/colab/FINAL_PIPELINE_REPORT.json', 'w') as f:
    json.dump(report, f, indent=2)

md_report = f"""# FX Sentiment Analysis Pipeline Report

**Execution Time**: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}
**Mode**: {'90-Day Smoke Test' if SMOKE_TEST else '15-Year Full Pipeline'}
**Date Range**: {os.environ.get('START_DATE')} to {os.environ.get('END_DATE')}


| Stage | Status |
|-------|--------|
| Bronze Ingestion | {'✅ SUCCESS' if bronze_success else '❌ FAILED'} |
| Silver Features | {'✅ SUCCESS' if silver_success else '❌ FAILED'} |
| Gold Matrix & Training | {'✅ SUCCESS' if training_success else '❌ FAILED'} |


- **Models**: `/content/models/` (MLflow format)
- **Metrics**: `/content/reports/colab/*.json`
- **Plots**: `/content/reports/colab/*.png`
- **Delta Tables**: `/content/delta/`


- ✅ Chronological splits maintained
- ✅ Proper time windows enforced
- ✅ Left-only joins with appropriate lags
- ✅ Event windows properly aligned


{'Pipeline completed successfully! Models are ready for deployment.' if training_success else 'Review failed stages and check logs for troubleshooting.'}
"""

with open('/content/reports/colab/FINAL_PIPELINE_REPORT.md', 'w') as f:
    f.write(md_report)

print("\n" + "="*60)
print("🎯 FINAL PIPELINE REPORT")
print("="*60)
print(md_report)
print("="*60)

print("\n📁 Output Files:")
!find /content -name "*.json" -o -name "*.png" -o -name "*.md" | head -20

if training_success:
    print("\n🎉 15-Year FX Sentiment Analysis Pipeline completed successfully!")
    print("📊 Ready for model deployment and backtesting.")
else:
    print("\n⚠️ Pipeline completed with errors. Review logs above.")